# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Sep 22 2022 1:27PM,246987,21,627599-1,"NBTY Global, Inc.",Released
1,Sep 22 2022 1:20PM,247077,19,60016935-1,"GUSA Granules USA, Inc.",Executing
2,Sep 22 2022 1:11PM,247076,10,CLI00387.1,"CLINUVEL, Inc.",Released
3,Sep 22 2022 12:57PM,247074,10,SONSB0001951,"Nextsource Biotechnology, LLC",Released
4,Sep 22 2022 12:31PM,247072,16,8556513,Sartorius Bioprocess Solutions,Released
5,Sep 22 2022 11:53AM,247061,10,8559682,"Akron BioProducts, LLC",Released
6,Sep 22 2022 11:53AM,247060,10,EYE20092022,Eye Pharma Inc,Released
7,Sep 22 2022 11:09AM,247048,20,8555178,"ACI Healthcare USA, Inc.",Released
8,Sep 22 2022 11:09AM,247048,20,8555180,"ACI Healthcare USA, Inc.",Released
9,Sep 22 2022 11:09AM,247048,20,8555196,"ACI Healthcare USA, Inc.",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
21,247061,Released,1
22,247072,Released,1
23,247074,Released,1
24,247076,Released,1
25,247077,Executing,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Executing,Released
id,,
247061,NaN,1.0
247072,NaN,1.0
247074,NaN,1.0
247076,NaN,1.0
247077,1.0,NaN


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Executing,Released
id,,
246977,0.0,13.0
246985,1.0,0.0
246987,0.0,1.0
246989,1.0,0.0
247005,4.0,7.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Executing,Released
id,,
246977,0,13
246985,1,0
246987,0,1
246989,1,0
247005,4,7


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Executing,Released
0,246977,0,13
1,246985,1,0
2,246987,0,1
3,246989,1,0
4,247005,4,7


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Executing,Released
0,246977,,13
1,246985,1,
2,246987,,1
3,246989,1,
4,247005,4,7


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Sep 22 2022 1:27PM,246987,21,"NBTY Global, Inc."
1,Sep 22 2022 1:20PM,247077,19,"GUSA Granules USA, Inc."
2,Sep 22 2022 1:11PM,247076,10,"CLINUVEL, Inc."
3,Sep 22 2022 12:57PM,247074,10,"Nextsource Biotechnology, LLC"
4,Sep 22 2022 12:31PM,247072,16,Sartorius Bioprocess Solutions
5,Sep 22 2022 11:53AM,247061,10,"Akron BioProducts, LLC"
6,Sep 22 2022 11:53AM,247060,10,Eye Pharma Inc
7,Sep 22 2022 11:09AM,247048,20,"ACI Healthcare USA, Inc."
42,Sep 22 2022 11:02AM,247052,10,ISDIN Corporation
49,Sep 22 2022 10:54AM,247043,10,Eywa Pharma Inc.


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Executing,Released
0,Sep 22 2022 1:27PM,246987,21,"NBTY Global, Inc.",,1
1,Sep 22 2022 1:20PM,247077,19,"GUSA Granules USA, Inc.",1,
2,Sep 22 2022 1:11PM,247076,10,"CLINUVEL, Inc.",,1
3,Sep 22 2022 12:57PM,247074,10,"Nextsource Biotechnology, LLC",,1
4,Sep 22 2022 12:31PM,247072,16,Sartorius Bioprocess Solutions,,1
5,Sep 22 2022 11:53AM,247061,10,"Akron BioProducts, LLC",,1
6,Sep 22 2022 11:53AM,247060,10,Eye Pharma Inc,,1
7,Sep 22 2022 11:09AM,247048,20,"ACI Healthcare USA, Inc.",1,34
8,Sep 22 2022 11:02AM,247052,10,ISDIN Corporation,,7
9,Sep 22 2022 10:54AM,247043,10,Eywa Pharma Inc.,,2


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,Sep 22 2022 1:27PM,246987,21,"NBTY Global, Inc.",1,
1,Sep 22 2022 1:20PM,247077,19,"GUSA Granules USA, Inc.",,1
2,Sep 22 2022 1:11PM,247076,10,"CLINUVEL, Inc.",1,
3,Sep 22 2022 12:57PM,247074,10,"Nextsource Biotechnology, LLC",1,
4,Sep 22 2022 12:31PM,247072,16,Sartorius Bioprocess Solutions,1,
5,Sep 22 2022 11:53AM,247061,10,"Akron BioProducts, LLC",1,
6,Sep 22 2022 11:53AM,247060,10,Eye Pharma Inc,1,
7,Sep 22 2022 11:09AM,247048,20,"ACI Healthcare USA, Inc.",34,1
8,Sep 22 2022 11:02AM,247052,10,ISDIN Corporation,7,
9,Sep 22 2022 10:54AM,247043,10,Eywa Pharma Inc.,2,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Sep 22 2022 1:27PM,246987,21,"NBTY Global, Inc.",1,
1,Sep 22 2022 1:20PM,247077,19,"GUSA Granules USA, Inc.",,1
2,Sep 22 2022 1:11PM,247076,10,"CLINUVEL, Inc.",1,
3,Sep 22 2022 12:57PM,247074,10,"Nextsource Biotechnology, LLC",1,
4,Sep 22 2022 12:31PM,247072,16,Sartorius Bioprocess Solutions,1,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


,Date,id,Warehouse,Customer,Released,Executing
0,Sep 22 2022 1:27PM,246987,21,"NBTY Global, Inc.",1.0,NaN
1,Sep 22 2022 1:20PM,247077,19,"GUSA Granules USA, Inc.",NaN,1.0
2,Sep 22 2022 1:11PM,247076,10,"CLINUVEL, Inc.",1.0,NaN
3,Sep 22 2022 12:57PM,247074,10,"Nextsource Biotechnology, LLC",1.0,NaN
4,Sep 22 2022 12:31PM,247072,16,Sartorius Bioprocess Solutions,1.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Sep 22 2022 1:27PM,246987,21,"NBTY Global, Inc.",1.0,0.0
1,Sep 22 2022 1:20PM,247077,19,"GUSA Granules USA, Inc.",0.0,1.0
2,Sep 22 2022 1:11PM,247076,10,"CLINUVEL, Inc.",1.0,0.0
3,Sep 22 2022 12:57PM,247074,10,"Nextsource Biotechnology, LLC",1.0,0.0
4,Sep 22 2022 12:31PM,247072,16,Sartorius Bioprocess Solutions,1.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
10,2223363,34.0,4.0
12,247040,1.0,0.0
15,494060,2.0,40.0
16,494114,5.0,0.0
19,494116,7.0,1.0
20,741114,36.0,2.0
21,740961,1.0,2.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,10,2223363,34.0,4.0
1,12,247040,1.0,0.0
2,15,494060,2.0,40.0
3,16,494114,5.0,0.0
4,19,494116,7.0,1.0
5,20,741114,36.0,2.0
6,21,740961,1.0,2.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,10,34.0,4.0
1,12,1.0,0.0
2,15,2.0,40.0
3,16,5.0,0.0
4,19,7.0,1.0
5,20,36.0,2.0
6,21,1.0,2.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,34.0
1,12,Released,1.0
2,15,Released,2.0
3,16,Released,5.0
4,19,Released,7.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,19,20,21
Status,,,,,,,
Executing,4.0,0.0,40.0,0.0,1.0,2.0,2.0
Released,34.0,1.0,2.0,5.0,7.0,36.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,19,20,21
0,Executing,4.0,0.0,40.0,0.0,1.0,2.0,2.0
1,Released,34.0,1.0,2.0,5.0,7.0,36.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,19,20,21
0,Executing,4.0,0.0,40.0,0.0,1.0,2.0,2.0
1,Released,34.0,1.0,2.0,5.0,7.0,36.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()